In [25]:
%reload_ext autoreload
%autoreload 2

# %gui qt

import time
from tqdm import tqdm
from pathlib import Path
import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import fileManagement as fm
import dbManagement as dbm

# import redCellGUI as rgui

In [6]:
# Immediate next steps for programming work:
# - ROICaT index alignment tools

In [2]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'
# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In [27]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [77]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In [28]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [8]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 1.14 s
Wall time: 476 ms


In [9]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [56]:
# Database Requirements: 
# ---------------------
# GUI: db manager
# - show full database -
# - show selected entries (like for example entries with imaging but no suite2p) -
#                --> It would be sweet to make this work where there are some defaults, but then there is also a "filter command" option which you type and goes as the input to the filtering
# - click on entry and do things:
#                --> open file explorer to that session
#                --> do suite2p
#                --> do red cell management
# - update table data? 
# ---------------------
# Operational Commands: 
# - Automatically do suite2p 
# - Check if registration was done before a suite2p update
# - Find way to determine if a suite2p file was updated after the folder was made (indicating that QC was done afterwards) -- so registration should be rerun for imaging data at least
#    -- check if "iscell" is the only thing that was done after registration. In this case, just update the iscell oneData
# ---------------------
# - Add new entry GUI!!!
# ---------------------
# - Check session folders to see if new session is present that isn't registered in the database? 
# ---------------------
# vrExperimentUpdates: 
# -- if edited iscell, will need to re-register the number of ROIs per plane (in "self.values") and reprocess imaging data!
# ---------------------
# vrExperimentRegistration
# -- if error, then note there was an error and append an explanation...!!!
# ---------------------
# I need a readme file for the dbManagement component that shows common uses (e.g. a tutorial)
#    - because otherwise I'll forget what I wrote....
# ---------------------

In [39]:
def checkMethod(*inputs, hello=None, **kwargs):
    for i in inputs: print(i)
    print(hello)
    for k,v in kwargs.items(): print(k,v)
    
checkMethod('hello','world',yep='hi',nope='nope',hello='hi')

hello
world
hi
yep hi
nope nope


In [ ]:
# Creating a suite2p classifier
# Let's put a few sessions (all planes) into the classifier by "building" a new classifier
# I'll get at least 2 sessions from each mouse...

# List of QC'd sessions:
# ATL012/2023-01-18/701 Planes 
# ATL028/2023-08-08/701 Planes 1, 2

In [26]:
vrdb = dbm.vrDatabase()

In [85]:
db = vrdb.getTable(suite2p=True)
for idx, row in db.iterrows():
    print(vrdb.vrSession(row).sessionPrint(), row['suite2pDate'])
          

ATL012/2023-01-18/701 2023-01-18 14:20:04
ATL012/2023-01-20/701 2023-01-20 11:15:30
ATL012/2023-01-20/702 2023-01-20 13:44:47
ATL012/2023-01-23/701 2023-01-23 13:32:56
ATL012/2023-01-24/703 2023-01-24 11:05:34
ATL012/2023-01-25/701 2023-01-25 10:19:05
ATL012/2023-01-26/702 2023-08-10 09:23:42
ATL012/2023-01-27/701 2023-01-27 12:41:10
ATL012/2023-01-30/702 2023-08-17 17:37:21
ATL012/2023-02-01/701 2023-02-11 18:52:22
ATL012/2023-02-02/701 2023-02-12 09:54:55
ATL012/2023-02-06/701 2023-02-12 14:36:14
ATL012/2023-02-07/701 2023-02-12 17:54:09
ATL012/2023-02-08/702 2023-02-13 14:42:00
ATL012/2023-02-09/701 2023-02-17 13:24:17
ATL012/2023-02-14/703 2023-02-17 14:44:52
ATL012/2023-02-15/701 2023-02-17 18:45:50
ATL012/2023-02-16/701 2023-02-20 11:05:53
ATL012/2023-02-20/701 2023-02-20 16:24:37
ATL012/2023-02-21/702 2023-02-27 09:20:44
ATL012/2023-02-22/701 2023-02-27 10:11:00
ATL012/2023-02-27/701 2023-02-27 14:07:55
ATL012/2023-02-28/701 2023-03-22 11:00:00
ATL012/2023-03-01/701 2023-08-17 2

In [79]:
vrdb.printRegistrationErrors()

In [84]:
vrdb.updateSuite2pDateTime()
vrdb.checkS2P(withDatabaseUpdate=True)

In [24]:
vre.vrSession('ATL012','2023-01-18','701').sessionPath()

WindowsPath('C:/Users/andrew/Documents/localData/ATL012/2023-01-18/701')

In [75]:
vrdb.printRequiresS2P(needsQC=True)

Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-18/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-20/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-20/702
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-23/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-24/703
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-25/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-26/702
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-27/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-30/702
Database indicates that suite2p has been run but not QC'd: ATL012/2023-02-01/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-02-02/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-02-06/701
Database indicates that suit